# Comparación de los algoritmos implementados desde 0 con los algoritmos tomados de bibliotecas. 

#### Bibliotecas

In [1]:
# Bibliotecas públicas
import random
import os
from sklearn import datasets

# Bibliotecas personalizadas
from Scripts.dataset import open_dataset
from Scripts.knn import classify as knn_classify
from Scripts.evaluation_metrics import *
from Scripts.confusion_matrix import confusion_matrix

### Cargando los datos

In [2]:
# Se especifican las rutas al conjunto de entrenamiento y prueba
dataset1_train = open_dataset("Datasets/Datos-S-Entrena.txt", True)
dataset1_test = open_dataset("Datasets/S-Prueba.txt", True)

# Conjunto de datos unido (para validación cruzada)
dataset1_completo = dataset1_train + dataset1_test

***************************************************
Cantidad de ejemplos en el conjunto:  397
Cantidad de atributos: 18
Ejemplo de instancia:  ([140.0, 125.0, 0.0, 0.0, 0.0, 0.2777779, 0.06296301, 0.66666675, 0.31111118, 6.185185, 7.3333335, 7.6666665, 3.5555556, 3.4444444, 4.4444447, -7.888889, 7.7777777, 0.5456349], 0)
***************************************************
Cantidad de ejemplos en el conjunto:  23
Cantidad de atributos: 18
Ejemplo de instancia:  ([242.0, 65.0, 0.0, 0.0, 0.0, 0.5555553, 0.27216578, 1.0, 0.42163682, 8.925926, 6.111111, 14.333333, 6.3333335, -8.444445, 16.222221, -7.7777777, 14.333333, 0.5834605], 6)


### Realizando predicciones con conjuntos de entrenamiento y prueba

#### k vecinos más cercanos

In [3]:
# Prediciendo sobre el conjunto de prueba
knn_predictions = knn_classify(dataset1_train, dataset1_test, k=3, metric='euclidean')
print("Exactitud del clasificador: ", str(accuracy(knn_predictions, False)*100)+"%")

Exactitud del clasificador:  91.30434782608695%


In [4]:
# Imprimiento la matriz de confusión
confusion_matrix(dataset1_completo,knn_predictions)

[4, 3, 4, 3, 4, 3, 2]
		[ 0 ,   1 ,   2 ,   3 ,   4 ,   5 ,   6 ]


### Realizando predicciones con validación cruzada

#### k vecinos más cercanos

In [5]:
# Aleatorizando el orden del conjunto de datos
random.shuffle(dataset1_completo)

# Realizando validación cruzada
# k_fold_cross_val(data, n_folds, k_vecinos, metric)
k_fold_scores = k_fold_cross_val(dataset1_completo, 10, 8, 'euclidean')
print("Puntajes por fold:", k_fold_scores)
print("Promedio de precisión en k-folds:", sum(k_fold_scores)/len(k_fold_scores))

Diviendo los datos en 10 folds de tamaño 42
Puntajes por fold: [0.8809523809523809, 0.9047619047619048, 0.9285714285714286, 0.9523809523809523, 0.9285714285714286, 0.8333333333333334, 0.9047619047619048, 0.9285714285714286, 0.8571428571428571, 0.8333333333333334]
Promedio de precisión en k-folds: 0.8952380952380954
